# Número de Casos por CC.AA. hasta el día actual

In [20]:
import pandas as pd
import io
import requests
from datetime import date

url_general = 'https://covid19.isciii.es/resources/data.csv'
url_ccaa = 'https://covid19.isciii.es/resources/ccaa.csv'

s = requests.get(url_general).content
general = pd.read_csv(io.StringIO(s.decode('utf-8')))

t = requests.get(url_ccaa).content
cc_aa = pd.read_csv(io.StringIO(t.decode('utf-8')))

In [21]:
general

,Fecha,Hora,Casos,Defunciones,Recuperados
0,19 de marzo de 2020,20:00,19980,1002,1588


In [22]:
cc_aa

,CCAA,Nº Casos
0,Andalucía,1287
1,Aragón,360
2,Asturias,344
3,Baleares,203
4,Canarias,287
5,Cantabria,144
6,Castilla La Mancha,1044
7,Castilla y León,1147
8,Cataluña,3270
9,Ceuta,5


In [23]:
hoy = date.today().strftime('%d/%m/%Y')

In [24]:
import csv
general.to_csv(date.today().strftime('%Y%m%d') + '_general.csv')
cc_aa.to_csv(date.today().strftime('%Y%m%d') + '_ccaa.csv')

In [25]:
cc_aa=cc_aa.sort_values(by='Nº Casos', ascending=True)
ccaa = cc_aa['CCAA']
casos = cc_aa['Nº Casos']

In [26]:
casos

9        5
14      24
5      144
3      203
15     204
4      287
11     297
2      344
1      360
18     497
16     554
12     578
6     1044
10    1105
7     1147
0     1287
17    1465
8     3270
13    7165
Name: Nº Casos, dtype: int64

In [27]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Bar(
    x=casos,
    y=ccaa,
    text=casos,
    textposition='outside',
    orientation='h',
    name="casos_ccaa"
))

fig.update_layout(
    title="Número de casos por Comunidades Autónomas hasta el " + hoy,
    xaxis_title="Número casos",
    yaxis_title="Comunidades Autónomas",
    font=dict(
        family="Courier New, monospace",
        size=15,
        color="#000000"
    )
)

fig.show()

# Porcenatje variación casos por CC.AA desde el día anterior al actual

In [28]:
from datetime import datetime
from datetime import timedelta
import json

now = datetime.now()
new_date = now + timedelta(days=-1)
ayer = new_date.strftime('%Y%m%d')
ayer

'20200319'

In [35]:
 casos_dia_anterior = pd.read_csv(ayer + '_ccaa.csv', index_col=None)

In [36]:
casos_dia_anterior

,Unnamed: 0,CCAA,Nº Casos
0,0,Andalucía,1008
1,1,Aragón,281
2,2,Asturias,292
3,3,Baleares,169
4,4,Canarias,220
5,5,Cantabria,83
6,6,Castilla La Mancha,801
7,7,Castilla y León,868
8,8,Cataluña,2702
9,9,Ceuta,1


In [37]:
variacion_porcentaje = {}
variacion_absoluta = {}
for i in range(19):
    ccaa = cc_aa['CCAA'][i]
    casos = cc_aa['Nº Casos'][i]
    for j in range(19):
        ccaa_ayer = casos_dia_anterior['CCAA'][j]
        casos_ayer = int(casos_dia_anterior['Nº Casos'][j])
        if ccaa_ayer == ccaa:
            variacion = (int(casos) - int(casos_ayer))/int(casos_ayer) * 100
            variacion_porcentaje[cc_aa['CCAA'][i]] = str(round(variacion, 2))
            variacion_absoluta[cc_aa['CCAA'][i]] = variacion
variacion_porcentaje

{'Andalucía': '27.68',
 'Aragón': '28.11',
 'Asturias': '17.81',
 'Baleares': '20.12',
 'Canarias': '30.45',
 'Cantabria': '73.49',
 'Castilla La Mancha': '30.34',
 'Castilla y León': '32.14',
 'Cataluña': '21.02',
 'Ceuta': '400.0',
 'C. Valenciana': '19.98',
 'Extremadura': '23.24',
 'Galicia': '27.59',
 'Madrid': '5.73',
 'Melilla': '4.35',
 'Murcia': '22.16',
 'Navarra': '14.94',
 'País Vasco': '23.11',
 'La Rioja': '6.2'}

In [38]:
tabla_variacion = pd.DataFrame([[key, variacion_porcentaje[key]] for key in variacion_porcentaje.keys()], columns=['CCAA', '% Variacion'])

In [39]:
tabla_variacion

,CCAA,% Variacion
0,Andalucía,27.68
1,Aragón,28.11
2,Asturias,17.81
3,Baleares,20.12
4,Canarias,30.45
5,Cantabria,73.49
6,Castilla La Mancha,30.34
7,Castilla y León,32.14
8,Cataluña,21.02
9,Ceuta,400.0


In [40]:
comunidades = tabla_variacion['CCAA']
var = tabla_variacion['% Variacion']

clrrojo = '#ff0000'
clrverde = '#00ff08'
clrs  = [clrverde if float(var[x]) < 0 else clrrojo for x in range(len(var))]

fig2 = go.Figure()

fig2.add_trace(go.Bar(
    x=comunidades,
    y=var,
    textposition='outside',
    text=var + '%',
    marker=dict(color=clrs),
    name="variacion_ccaa"
))

fig2.update_layout(
    title="Variacion (%) del número de casos por CC.AA. respecto a ayer",
    xaxis_title="Comunidades Autónomas",
    yaxis_title="% Variación",
    font=dict(
        family="Courier New, monospace",
        size=15,
        color="#000000"
    )
)

fig2.show()